# Common<br>
Run this sect first

In [ ]:
def dlpromptexample():
  !wget https://github.com/TabuaTambalam/DalleWebms/releases/download/0.1/pexmp.7z
  !7z x pexmp.7z
  !wget -O web/psg1.htm https://raw.githubusercontent.com/TabuaTambalam/DalleWebms/main/docs/web/psg1.htm
  !wget -O web/svr.py https://raw.githubusercontent.com/TabuaTambalam/DalleWebms/main/docs/web/svr.py
  !wget -P web/ https://github.com/TabuaTambalam/tulalia/releases/download/1.1/diffusion_emb_pnnx.pt
  !wget -P web/ https://github.com/TabuaTambalam/tulalia/releases/download/1.1/diffusion_mid_pnnx.pt
  !wget -P web/ https://github.com/TabuaTambalam/tulalia/releases/download/1.1/diffusion_out_pnnx.pt
  !wget https://huggingface.co/Larvik/tfmod/resolve/main/sgmk.7z
  !wget https://raw.githubusercontent.com/TabuaTambalam/DalleWebms/main/docs/sd/BERTEmbedder.py
  !wget https://github.com/TabuaTambalam/DalleWebms/releases/download/0.1/ldms.7z
  !7z x ldms.7z
  !mv ldm_xfm ldm
  !7z e sgmk.7z
  !rm *.7z
  !pip install ftfy transformers omegaconf triton==2.0.0.dev20220701 einops accelerate
  !wget https://raw.githubusercontent.com/TabuaTambalam/DalleWebms/main/docs/sd/jkt.py



import os
from threading import Thread
if not os.path.isfile('PromptFuncsExample/MultiPrompt_average.txt'):
  t3 = Thread(target = dlpromptexample)
  a3 = t3.start()
!mv web/svr.py_one web/svr.py
!rm /content/sample_data/izh.txt

In [ ]:
import os
import numpy as np
from PIL import Image
import PIL
import torch
import time

torch.set_grad_enabled(False)
torch.set_num_threads(os.cpu_count())
torch.backends.cudnn.enabled = True
torch.backends.cudnn.allow_tf32 = True
torch.backends.cudnn.benchmark = True
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cuda.matmul.allow_fp16_reduced_precision_reduction = True

def procckpt():
  os.link(alter_path,'tojit.ckpt')
  if not os.path.isfile('ck2jit.py'):
    !pip install pytorch-lightning torch-fidelity clip-anytorch kornia
    !wget https://raw.githubusercontent.com/TabuaTambalam/DalleWebms/main/docs/sd/ck2jit.py
    !wget https://github.com/TabuaTambalam/tulalia/releases/download/1.1/freqs.npy
  waitingpip()
  !mkdir ckpts
  !python ck2jit.py
  !rm tojit.ckpt
  realname=alter_path.replace('.','/').split('/')[-2]
  !mv ckpts {realname}
  return realname+'/'

def f_dljit(ver='470k',dfsver=''):
  convckpt=False
  if ver == 'ckpt':
    convckpt=True
    ver = '840k'
  if not os.path.isfile('imgencoder_pnnx.pt'):
    !wget https://huggingface.co/Larvik/sd{ver}/resolve/main/alphas_cumprod.npz
    !wget https://huggingface.co/Larvik/tfmod/resolve/main/transformer_pnnx.pt
    !wget https://huggingface.co/Larvik/sd{ver}/resolve/main/autoencoder_pnnx.pt
    !wget https://huggingface.co/Larvik/sd{ver}/resolve/main/imgencoder_pnnx.pt
  if convckpt:
    return procckpt()
  ver+=dfsver
  !mkdir {ver}
  if not os.path.isfile(ver+'/diffusion_out_pnnx.pt'):
    !nohup wget -P {ver}/ https://huggingface.co/Larvik/sd{ver}/raw/main/convd.txt &
    !nohup wget -P {ver}/ https://huggingface.co/Larvik/sd{ver}/resolve/main/diffusion_emb_pnnx.pt &
    !wget -P {ver}/ https://huggingface.co/Larvik/sd{ver}/resolve/main/diffusion_mid_pnnx.pt
    !wget -P {ver}/ https://huggingface.co/Larvik/sd{ver}/resolve/main/diffusion_out_pnnx.pt
  return ver+'/'

def get_keys_to_submodule(model):
  keys_to_submodule = {}
  # iterate all submodules
  for submodule_name, submodule in model.named_modules():
      # iterate all paramters in each submobule
      for param_name, param in submodule.named_parameters():
          # param_name is organized as .. ...
          splitted_param_name = param_name.split('.')
          # we cannot go inside it anymore. This is the actual parameter
          is_leaf_param = len(splitted_param_name) == 1
          if is_leaf_param:
              # we recreate the correct key
              key = f"{submodule_name}.{param_name}"
              # we associate this key with this submodule
              keys_to_submodule[key] = submodule
              
  return keys_to_submodule

def load_state_dict_with_low_memory(model, state_dict,modifyfunc=None,fill=True):
  if modifyfunc is not None:
    state_dict=modifyfunc(state_dict)
  print('======hacky load======')
  keys_to_submodule = get_keys_to_submodule(model)
  mste=model.state_dict()
  for key, submodule in keys_to_submodule.items():
      if key[0] == '.':
        key=key[1:]
      if key in state_dict:
        val = state_dict[key]
      elif fill:
        print(key)
        val = torch.ones(mste[key].shape, dtype= torch.float16)
      else:
        continue

      param_name = key.split('.')[-1]
      new_val = torch.nn.Parameter(val,requires_grad=False)
      setattr(submodule, param_name, new_val)


def load_img(path):
  if path.endswith('.npy'):
    return torch.tensor(np.load(path))
  elif os.path.isfile(path+'.npy'):
    return torch.tensor(np.load(path+'.npy'))
  
  image = Image.open(path).convert("RGB")
  w, h = image.size
  
  w2, h2 = map(lambda x: x - x % 64, (w, h))
  print(f"resize input image to size ({w2}, {h2}) from {path}")
  if w!=w2 or h!=h2:
    image = image.resize((w2, h2), resample=PIL.Image.LANCZOS)
  image = np.array(image).astype(np.float32) / 255.0
  image = image[None].transpose(0, 3, 1, 2)
  image = torch.from_numpy(image)
  rpt = 2.*image - 1.
  H=rpt.size(2)
  W=rpt.size(3)
  preimg=imgenc(  rpt, torch.randn(torch.Size([1,4,H>>3,W>>3]))  )*0.18215
  np.save(path+'.npy',preimg.numpy())
  return preimg

def make_ldm_unet():
  from accelerate import init_empty_weights
  from ldm.modules.diffusionmodules.openaimodel import UNetModel
  import jkt
  mkmodel_state_dict=jkt.mkmodel_state_dict
  in_chn=4
  sdt_func=None
  with init_empty_weights():
    ldm_unet = UNetModel(
            image_size=32,
            in_channels=in_chn,out_channels=4,
                model_channels=320,
                attention_resolutions=[4,2,1],
                num_res_blocks=2,
                channel_mult=[1,2,4,4],
                num_heads=8,
                use_spatial_transformer=True,
                context_dim=768,
                legacy= False).requires_grad_(False)
  load_state_dict_with_low_memory(ldm_unet,mkmodel_state_dict([diffusion_emb,diffusion_mid,diffusion_out]),modifyfunc=sdt_func)
  return ldm_unet

def calclim():
  rt=int((W*H)/0x1000)
  if rt > 3:
    return 0x800
  elif rt > 2:
    return 0x1000
  else:
    return 0x2000

def waitingpip():
  while not os.path.isfile('jkt.py'):
    time.sleep(1)

Lat_layer=3
def showlat(tok):
  ik=maalip[Lat_layer].permute(2,0,1)
  wyd=W<<1
  if Show_8heads:
    kole=[None]*8
    for i in range(8):
      img=ik[tok][i]
      mult=255/img.max()
      kole[i]=(img*mult).reshape(H>>2,W>>2).numpy().astype(np.uint8)
    im=np.hstack(kole)
    wyd=W<<4
  else:
    img=ik[tok].sum(0).reshape(H>>2,W>>2)
    mult=255/img.max()
    im=(img*mult).numpy().astype(np.uint8)
  im=Image.fromarray(im).resize((wyd, H<<1), resample=PIL.Image.LANCZOS)
  print(tok_dict[tok])
  display(im)

tok_dict=False
Show_8heads=False

# visualize

Only `(840k, Orig)` and `(470k, _a300)` are available, by now

In [ ]:
SDver='840k' #@param ['ckpt','470k','840k']
Dfm='Orig' #@param ['Orig','_a300']
alter_path='Opt: Put ckpt path here, then select [ ckpt ] in [ SDver ] (WIP)' #@param {type:'string'}
if Dfm=='Orig':
  Dfm=''
SDver=f_dljit(SDver,Dfm)
diffusion_emb = torch.jit.load(SDver+'diffusion_emb_pnnx.pt').eval()
diffusion_mid = torch.jit.load(SDver+'diffusion_mid_pnnx.pt').eval()
diffusion_out = torch.jit.load(SDver+'diffusion_out_pnnx.pt').eval()
imgenc=torch.jit.load('imgencoder_pnnx.pt').eval()
waitingpip()
import BERTEmbedder
ldm_unet=make_ldm_unet()
from einops import rearrange, repeat
from torch import nn, einsum

Inject logging func to qkv

In [ ]:
lim=0x2000
def qkv_opt(zself,q,k,v):
    h = zself.heads
    q, k, v = map(lambda t: rearrange(t, 'b n (h d) -> (b h) n d', h=h), (q, k, v))
    qsiz=q.shape[1]
    k=k.transpose(1,2)
    
    if qsiz > lim:
      out = []
      kk=0
      while kk < qsiz:
        kend=kk+lim
        if kend > qsiz:
          kend=qsiz    
        sim = torch.bmm(q[:,kk:kend],k) * zself.scale
        attn = sim.softmax(dim=-1)
        out.append(torch.bmm(attn, v))
        kk+=lim
      out=torch.cat(out,dim=1)
    else:
      sim = torch.bmm(q,k) * zself.scale
      attn = sim.softmax(dim=-1)
      if zself.attn_info==[1280,768]:
        maalip.append(attn)
      out = torch.bmm(attn, v)
    out = rearrange(out, '(b h) n d -> b n (h d)', h=h)
    return out

from ldm.modules.attention import CrossAttention_config
CrossAttention_config.qkv = qkv_opt

Opt: download an example image

In [ ]:
!wget -O lara.jpg https://pbs.twimg.com/media/FhAgJgjWQAEHICc.jpg

!wget https://raw.githubusercontent.com/justinpinkney/stable-diffusion/main/im-examples/Official_portrait_of_Barack_Obama.jpg
#  A photo of Barack Obama smiling with a big grin

!wget https://huggingface.co/datasets/Larvik/gelb/resolve/main/10_0A/001ae3b8.npy

Visualize attn heatmap<br>`Complex_Prompt`: Turn it on when there's stupid gigantic tag salad, or when using TI. 

In [ ]:
See_Image = 'lara.jpg' #@param {type:'string'}
See_concept = 'Lara croft, in rainforest, high definition, unreal engine, 4k' #@param {type:'string'}
Complex_Prompt=False #@param {type:'boolean'}
x_in=load_img(See_Image)
H, W = x_in.shape[-2:]
lim=calclim()
maalip=[]
t=torch.tensor([1e-7],dtype=torch.float)
if Complex_Prompt:
  cond_k, tok_dict = BERTEmbedder.encode_complex(See_concept)
else:
  cond_k, tok_dict = BERTEmbedder.encode(See_concept)
cond_v=cond_k

h, emb, hs = ldm_unet.forward_crossattn(x_in, t, cond_k,cond_v)   
h = ldm_unet.forward2(h, emb, cond_k, *hs[6:],cond_v) 
for i in range(len(tok_dict)):
  showlat(i)

Advanced Settings

In [ ]:
Show_8heads=False #@param {type:'boolean'}
Lat_layer=3 #@param {type:'integer'}
TxtEnc_early_return=0 #@param [0,-1,-2,-3,-4]
BERTEmbedder.transformer.cut=int(TxtEnc_early_return)

if tok_dict:
  for i in range(len(tok_dict)):
    showlat(i)

In [ ]:
BERTEmbedder.insert('<majipuri>')
BERTEmbedder.insert('<pekora>')

# srfixg

In [ ]:
import torch
srz=torch.load('srz.pt',map_location=torch.device('cpu'))

In [ ]:
hyb=srz[...,0]

In [ ]:
hyb=hyb[0]+3.2833

In [ ]:
img=(hyb*38.2).permute(1,2,0).numpy()

In [ ]:
img.shape

(128, 128, 3)

In [ ]:
import numpy as np
from PIL import Image

In [ ]:
Image.fromarray(img.astype(np.uint8), mode="RGB")